# Exploratory Analysis 

## **Imports**

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs
import tensorflow_data_validation as tfdv

## **Item Info**

In [2]:
item_info_df = pd.read_csv('item_info.csv', dtype={'item_no': str})
display(item_info_df.head())

,item_no,colour,gender_description,brand,product_group,min_age,max_age
0,206890150141030846,beige,unisex,bloomingville,furniture,NaN,NaN
1,7637494654837559066,pink,girls,petit bateau,clothing sets,0.125,2.0
2,7969520735315050609,beige,unisex,bobo choses,jumpers and knitwear,1.000,11.0
3,-565751122846696741,white,unisex,piupiuchick,tops,1.000,14.0
4,6688930722259797984,green,unisex,filibabba,baby feeding,NaN,NaN


In [3]:
item_info_statistics = tfdv.generate_statistics_from_dataframe(item_info_df )
tfdv.visualize_statistics(item_info_statistics)

## **User Info**

In [4]:
user_info_df = pd.read_csv('user_info.csv', dtype={'user_no': str})
user_info_df.head()

,user_no,country,aov,avg_markdown
0,8587933979694308845,sweden,479.2,-0.200000
1,2740387653650048572,sweden,837.6,-0.200000
2,2084988796719274722,sweden,942.4,-0.560000
3,-4662401680846085311,sweden,438.4,-0.620000
4,-1086148156436227367,sweden,664.8,-0.286667


In [5]:
user_info_statistics = tfdv.generate_statistics_from_dataframe(user_info_df)
tfdv.visualize_statistics(user_info_statistics)

## **User Item Interactions**

In [6]:
user_item_interaction_df = pd.read_csv('user_item_interactions.csv', dtype={'user_no': str, 'item_no': str})
display(user_item_interaction_df.head())

,user_no,item_no,date,eventtype
0,9060639138425951676,-478270421339298398,2021-06-24,purchased
1,9060639138425951676,2658388892627023500,2021-11-27,pageView
2,9060639138425951676,504233002877562247,2021-11-26,pageView
3,9060639138425951676,-3413566329152665076,2021-10-21,pageView
4,9060639138425951676,6700254580945881296,2021-10-10,pageView


In [7]:
interaction_statistics = tfdv.generate_statistics_from_dataframe(user_item_interaction_df)
tfdv.visualize_statistics(interaction_statistics)

In [11]:
print(user_item_interaction_df['date'].min())
print(user_item_interaction_df['date'].max())

2021-01-01
2021-11-30


---
## **Tasks**

 - [x] Get poetry working and add TFDV
 - [x] Profile each dataset
 - [x] User/item interactions details 
     - [x] How many page views are turned into purchases? 
     - [x] How many wishlist are turned into purchases
     - [x] Items per basket (big buyers?)
 - [ ] Join together 
     - [ ] Popular item types? Popular colors? Popular brands? 
 - [ ] KPIs 
     - [ ] Visitors per month
     - [ ] Conversion ratio
     - [ ] Total items sold (per month)
     - [ ] Sessions per month
     - [ ] Sessions with a purchase per month
     - [ ] Returned users 
 - [ ] Individual User Journey 
     - [ ] Dashboard
 - [ ] Cold start, new users/items
 - [ ] Train/test split and baseline
 - [ ] Determination of features
 - [ ] Write out data for training
 - [ ] Clean up and write text (summarize)
     - [ ] How many unique items/users
     - [ ] Most popular items, most frequent users
     - [ ] Event types
 
<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>

<div class="alert alert-block alert-success">
<b>Up to you:</b> Use green boxes sparingly, and only for some specific 
purpose that the other boxes can't cover. For example, if you have a lot 
of related content to link to, maybe you decide to use green boxes for 
related links from each section of a notebook.
</div>

<div class="alert alert-block alert-danger">
<b>Just don't:</b> In general, avoid the red boxes. These should only be
used for actions that might cause data loss or another major issue.
</div>

<div class="alert alert-block alert-warning">
<b>Example:</b> Use yellow boxes for examples that are not 
inside code cells, or use for mathematical formulas if needed.
    
```python
class Dev:
    def __init__(self):
```
</div>

$Hello_p$

---
## **User-Item Interaction Details**

 - [x] Purchases that weren't previously viewed or added to cart
 - [x] Page views/wishlist that turned into purchases
 - [x] Items per basket 

In [31]:
user_item_interaction_jan = user_item_interaction_df.loc[
    user_item_interaction_df['date'].apply(lambda x: x[:7]) == '2021-01']

In [32]:
user_item_interaction_jan

,user_no,item_no,date,eventtype
54,-4992480809564769711,8348016184428027188,2021-01-26,pageView
82,-9047880956100698698,6121207426465721414,2021-01-29,pageView
133,4197025502585486837,-6172044358363580016,2021-01-21,pageView
257,-1603776270012168559,6925252469515807917,2021-01-28,addToCart
258,-1603776270012168559,6194399194013600425,2021-01-28,addToCart
...,...,...,...,...
1011170,2740387653650048572,2559370552162586247,2021-01-17,addToCart
1011195,7068348551466544329,1472365912138241045,2021-01-19,purchased
1011196,7068348551466544329,981298674982707936,2021-01-19,purchased
1011197,7068348551466544329,5972339127440735267,2021-01-19,purchased


In [33]:
user_item_interaction_jan_groups = user_item_interaction_jan.groupby(['user_no', 'item_no'])

In [113]:
pivoted = pd.pivot_table(user_item_interaction_jan, 
                         index=['user_no', 'item_no'], 
                         columns=['eventtype'], 
                         aggfunc='count', 
                         fill_value=0).reset_index()

pivoted.columns = ['user_no', 'item_no', 'addToCart', 'pageView', 'purchased']

In [122]:
purchased_no_page_view = pivoted.loc[(pivoted['purchased'] == 1) & (pivoted['pageView'] == 0)]

In [129]:
pivoted = pd.pivot_table(user_item_interaction_df, 
                         index=['user_no', 'item_no'], 
                         columns=['eventtype'], 
                         aggfunc='count', 
                         fill_value=0).reset_index()

pivoted.columns = ['user_no', 'item_no', 'addToCart', 'addToWishlist', 'pageView', 'purchased']

In [130]:
pivoted

,user_no,item_no,addToCart,addToWishlist,pageView,purchased
0,-1000173980966705625,2356442837089120035,0,0,1,1
1,-1000173980966705625,6194820387173010563,0,0,1,0
2,-1000173980966705625,6283078728673886163,0,0,1,1
3,-1000190704253701773,383205862898605182,1,0,1,0
4,-1001470904269092144,-4036395326208857801,2,0,2,0
...,...,...,...,...,...,...
784712,9999165590351299,-7779279685545513760,0,0,1,0
784713,9999165590351299,169913927963481109,0,0,1,0
784714,9999165590351299,3042466209036108905,0,0,1,0
784715,99995943117292705,-8708142935431931645,0,0,0,1


In [136]:
purchases = pivoted.loc[pivoted['purchased'] == 1]
purchases

,user_no,item_no,addToCart,addToWishlist,pageView,purchased
0,-1000173980966705625,2356442837089120035,0,0,1,1
2,-1000173980966705625,6283078728673886163,0,0,1,1
17,-1001697075369787517,-1365382699650566863,1,0,0,1
19,-1001697075369787517,-7009163308018422334,1,1,1,1
20,-1001697075369787517,-752701044660672673,1,1,2,1
...,...,...,...,...,...,...
784698,999541357744859146,594778003920049884,0,0,0,1
784701,999541357744859146,7793588584802914643,0,0,0,1
784702,999541357744859146,8693370834128739003,0,0,0,1
784706,99973748111773990,-9172288605754916860,0,0,1,1


In [168]:
purchased = pivoted.loc[pivoted['purchased'] == 1]

purchased_no_other_events = purchased.loc[(purchased['addToCart'] == 0) & 
                                          (purchased['addToWishlist'] == 0) & 
                                          (purchased['pageView'] == 0)]

In [171]:
print(f"{len(purchased_no_other_events) / len(purchased):.2%} of purchases had no other events")

58.22% of purchases had no other events


<div class="alert alert-block alert-warning">
<b>Warning:</b> Many purchases have no other events, implications for how we track conversions, e.g. just because few pageViews turn into purchases isn't necessarily bad if we have a bunch of other purchases from unknown provenance.
</div>

**pageView, addToCard, addToWishlist** turned into purchased

In [164]:
def calc_percent_conversion(df, event):
    event_df = df.loc[df[event] > 0]
    purchases = event_df.loc[event_df['purchased'] > 0]
    return len(purchases) / len(event_df)

def print_conversion(df, event):
    conversion_ratio = calc_percent_conversion(df, event)
    print(f"{conversion_ratio:<10.2%} {event:<15} events resulted in a purchase")

In [166]:
print_conversion(pivoted, 'addToCart')
print_conversion(pivoted, 'addToWishlist')
print_conversion(pivoted, 'pageView')

22.29%     addToCart       events resulted in a purchase
8.37%      addToWishlist   events resulted in a purchase
12.74%     pageView        events resulted in a purchase


---

**Items per basket**

In [181]:
purchases = user_item_interaction_df.loc[user_item_interaction_df['eventtype'] == 'purchased']
session_baskets = purchases.groupby(['user_no', 'date']).agg('count').reset_index()[['user_no', 'date', 'item_no']]

In [183]:
session_baskets.describe()

,item_no
count,85452.000000
mean,2.658393
std,2.190587
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,20.000000


---

## **Join Together**

- [ ] Join together 
     - [ ] Popular item types? Popular colors? Popular brands? 